# Purpose of this notebook

This notebook produced the numpy arrays required for ml_test.ipynb from 20 second labeled pcaps. 

In [1]:
import os
import numpy as np
import re
from scapy.all import *
from datetime import datetime as dt
from matplotlib import pyplot as plt

/Users/michaelkearney/.local/share/virtualenvs/UNB_Datasets-qeLdalLc/lib/python3.9/site-packages/scapy/layers/ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/Users/michaelkearney/.local/share/virtualenvs/UNB_Datasets-qeLdalLc/lib/python3.9/site-packages/scapy/layers/ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


In [2]:
TIMEFRAME = 20          # Number of seconds per captured frame

PATH = "../pcap_data"           # Where the clips are saved

MIN_PACKETS = 50

CLIENTS = [
    "10.8.8.178",
    "10.8.8.130",
    "10.8.0.10",
    "10.8.0.14",
    "10.8.8.138",
    "10.8.8.234",
    "10.8.0.6",
    "10.8.8.134",
    "10.8.8.222",
    "10.8.8.246",
    "131.202.240.150",
    "131.202.240.242",
    "131.202.240.183",
    "131.202.240.150",
    "131.202.240.87",
    "131.202.240.101",
    "10.0.2.15",
    "192.168.254.0"
]
                          # List of Client IP addresses

In [3]:
dir_list = os.listdir(PATH)
clips = [os.path.join(PATH, clip) for clip in dir_list if clip[-5:] == ".pcap"]

In [4]:
def extract_features_from_clips(clip):

    matrix = np.zeros([TIMEFRAME * 10, 150, 2])
    start_time = clip[0].time
    for pkt in clip:
        if IP in pkt:
            length = (lambda x: x if x <= 1500 else 1500)(pkt[IP].len) # Packets over 1500 are rounded down to 1500
            dir = (lambda x: 0 if x[IP].src in CLIENTS else 1)(pkt)
            time_round = round(pkt.time - start_time, 1) 
            matrix[int(time_round * 10)-1][int(length / 10)-1][dir] += 1
    return np.array(matrix)
    
    

In [5]:
clips_list = [rdpcap(os.path.join(PATH, file)) for file in dir_list if file[-5:] == ".pcap"]

In [6]:
features_list = []
labels_list =[]

for idx, clip in enumerate(clips_list):
    if len(clip) > MIN_PACKETS:
        features_list.append(extract_features_from_clips(clip))
        label = clips[idx][len(PATH)+1:].split("-")[0]
        # label = re.split("\d", label)[0]   # Uncomment for use with ICSX dataset
        # label = re.split("_[AB]", label)[0]    # Uncomment for use with ICSX dataset
        label = label.split("-")[0]
        labels_list.append(label)


In [7]:
features = np.array(features_list)
labels = np.array(labels_list)

print(f"features shape {features.shape}\nlabels shape {labels.shape}")

features shape (3000, 200, 150, 2)
labels shape (3000,)


In [8]:
unique, counts = np.unique(labels, return_counts=True)

counts = np.asarray((unique, counts)).T
counts

array([['www.amazon.co.uk', '307'],
       ['www.bbc.co.uk', '269'],
       ['www.ebay.co.uk', '297'],
       ['www.facebook.com', '286'],
       ['www.google.co.uk', '301'],
       ['www.google.com', '310'],
       ['www.twitter.com', '290'],
       ['www.uwe.ac.uk', '320'],
       ['www.wikipedia.com', '308'],
       ['www.youtube.com', '312']], dtype='<U21')

In [9]:
# save = input("Do you want to save? y/n") == "y"
save = True

if save:
    date_time_format = '%Y_%m_%d__%H_%M_%S'
    current_datetime = dt.now()
    current_datetime_string = dt.strftime(current_datetime, date_time_format)

    with open(f"data/my_{TIMEFRAME}_sec_features-{current_datetime_string}.npy", "wb") as f:
        np.save(f, features)

    with open(f"data/my_{TIMEFRAME}_sec_labels-{current_datetime_string}.npy", "wb") as f:
        np.save(f, labels)